In [ ]:
# use this for cuda 12.x 
%pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
# use this for cuda 11.x
# !pip install onnxruntime-gpu
%pip install fastembed-gpu
%pip install torch

In [ ]:
%pip install qdrant_client
%pip install llama-index llama-index-core llama-index-llms-ollama llama_index.embeddings.ollama llama-index-embeddings-fastembed llama-index-readers-file 
%pip install llama-index-vector-stores-qdrant==0.2.10
#%pip install llama-index --upgrade --no-cache-dir --force-reinstall

In [2]:
# user settings
import os
user_settings = {
    'documents_path': '/home/micheldumontier/code/gba-rag/data/www.g-ba.de/beschluesse/',
    'file_extension': ['.htm','.pdf'],
    'exclude_directories': ['letzte-aenderungen'],
    
    'llm_model': 'llama3',
    'llm_temperature': 0.0,
    'llm_context_window': 4096,
    'llm_num_output': 512,
    
    'vectordb_url': 'http://localhost:6333',
    'vectordb_collection_name':'gba',
    'vectordb_api_key': "TOCHANGE"
}
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
settings = dotdict(user_settings)


In [3]:
import sys
import os 
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_dir = os.path.join(parent_dir, 'src')
sys.path.append(src_dir)


In [4]:
from typing import Any, Dict, List
import qdrant_client
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.qdrant import QdrantVectorStore
#from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter

# initialize the LLM model
llm = Ollama(
    model=settings.llm_model, 
    temperature=settings.llm_temperature, 
    device="cuda")

# select the embedding model
#embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
#from fastembedembedding import FastEmbedEmbedding 
import fastembedembedding 
embed_model = fastembedembedding.FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5", device="cuda")

try:
    client =  qdrant_client.QdrantClient(
        settings.vectordb_url,
        api_key=settings.vectordb_api_key)
            
    # select the local qdrant vector store and collection name
    vector_store = QdrantVectorStore(
        client = client,
        collection_name=settings.vectordb_collection_name
    )
except Exception as e:
    print(e)

# set the global settings
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.vector_store = vector_store
Settings.context_window = settings.llm_context_window
Settings.num_output = settings.llm_num_output
Settings.splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=50),	

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/home/micheldumontier/code/gba-rag/.venv/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


In [5]:
# check that we have CUDA support 
# output should be:
# ['CUDAExecutionProvider', 'CPUExecutionProvider']
from fastembed import TextEmbedding
embedding_model_gpu = TextEmbedding(
    model_name="BAAI/bge-small-en-v1.5", 
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"],
    device="cuda"
)
embedding_model_gpu.model.model.get_providers()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

['CUDAExecutionProvider', 'CPUExecutionProvider']

In [6]:
from llama_index.core.ingestion import IngestionPipeline

transformations = [TokenTextSplitter(chunk_size=1000, chunk_overlap=50), embed_model]
Settings.node_parser = settings.splitter
Settings.text_splitter = settings.splitter
Settings.transformations = transformations

pipeline = IngestionPipeline(
    transformations=transformations,
    vector_store=vector_store
)

#pipeline.persist("./pipeline_storage")


In [7]:
# here we collect all the HTML files 
import glob 
import os

def get_files_recursively(directory, extensions, exclude_dirs):
    files_list = []
    for root, dirs, files in os.walk(directory):
        # Remove excluded directories from the list of directories to walk
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        for file in files:
            if any(file.endswith(ext) for ext in extensions):
                files_list.append(os.path.join(root, file))
    return files_list

directory_to_search = settings.documents_path
file_extensions = settings.file_extension
exclude_directories = settings.exclude_directories
files = get_files_recursively(
    directory_to_search, file_extensions, exclude_directories)

# Print the number of files found
print(len(files))

10565


In [8]:
### functions to load the HTML file, extract the PDF files , and also load the PDF files
import os
from tqdm import tqdm
from bs4 import BeautifulSoup
from llama_index.core import SimpleDirectoryReader

def get_meta(file_path):
    return {"file_path": file_path}

def getPDFFilesFromHTML(html_text):
    pdfdocs = []
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Get the download section
    gba_download_list = soup.find_all('ul', class_='gba-download-list')
    
    for ul in gba_download_list:
        for li in ul.find_all('li'):
            span_tags = li.find_all('span', class_='gba-download__text')
            pdf_type = span_tags[0].text.strip()
            
            pdf_link = li.find_all('a', href=lambda href: href and href.endswith('.pdf'))
            file_path = pdf_link[0]['href']
            
            pdf_path = os.path.dirname(file_path)+"/"
            pdf_path = pdf_path.replace('../../','/')
            pdf_path = pdf_path + os.path.basename(file_path)
            
            pdfdocs.append({'section': pdf_type, 'file': pdf_path})
    return pdfdocs        

def loadPDFDocs(html_file, pdfdocs, path):
    pdf_documents = []
    
    #for pdf_path in tqdm( pdfdocs, desc="Loading PDF files"):
    for pdf_path in pdfdocs:
        file = path+pdf_path['file'] 
        # check if file exists
        if not os.path.exists(file):
            print(f"File {file} does not exist")
            continue
        
        pdfloader = SimpleDirectoryReader(input_files=[file], file_metadata=get_meta)
        pdfloaded_docs = pdfloader.load_data()
                
        ## add the metadata
        if len(pdfloaded_docs) > 0:
            # log(f"Loaded {len(pdfloaded_docs)} items from pdf files")
            for pdfdoc in pdfloaded_docs:
                sub_file_path = os.path.join(*html_file.split(os.sep)[6:])
                
                pdfdoc.metadata['from_html'] = sub_file_path
                pdfdoc.metadata['document_type']  = 'attachment'
                pdfdoc.metadata['file_type']      = 'pdf'
                pdfdoc.metadata['section']        = pdf_path['section']
                
                sub_file_path = os.path.join(*file.split(os.sep)[6:])
                pdfdoc.metadata['file_path'] = sub_file_path
                
                pdf_documents.append(pdfdoc)    
                
    return pdf_documents

def remove_progress_file(filename):
    if os.path.exists(filename):
        os.remove(filename)
        
def save_progress_to_file(digit, filename):
    # Save the single digit to a file
    with open(filename, 'w') as file:
        file.write(str(digit))
        
def load_progress_from_file(filename):
    # Load the single digit from a file
    if not os.path.exists(filename):
        print(f"File '{filename}' does not exist.")
        return 0
    
    with open(filename, 'r') as file:
        digit = file.read()
    return int(digit)


In [33]:
# now read each html file and get the PDF names
#from utils import getPDFFilesFromHTML, loadPDFDocs, remove_progress_file, load_progress_from_file, save_progress_to_file, get_meta
from llama_index.core import SimpleDirectoryReader
from tqdm import tqdm

progress_file = "progress.txt"
max_file_to_process = 10
restart = False

progress = load_progress_from_file(progress_file)
if restart == True:
    remove_progress_file(progress_file)
    progress = 0

html_documents = []
i = 0
#for index, file in enumerate(files):    
for html_file in tqdm(files, desc="Processing HTML files"):
    i += 1
    if i <= progress:
        continue
    
    reader = SimpleDirectoryReader(input_files=[html_file], file_metadata=get_meta)
    html_docs = reader.load_data()
    if(html_docs == []):
        print(f"Could not load {html_file}")
        continue
    
    html_doc = html_docs[0]
    html_doc.metadata['filetype']='html'
    html_doc.metadata['document_type']='gba_study'
    html_documents.append(html_doc)
    
    pdfdocs = getPDFFilesFromHTML(html_doc.text)
    
    if(pdfdocs != []):
        path = os.path.join('/',*settings.documents_path.split(os.sep)[:-2])
        pdf_documents = loadPDFDocs(html_file, pdfdocs, path)
        if (len(pdf_documents) > 0):
            
            nodes = pipeline.run(documents=pdf_documents)
            #nodes = await pipeline.arun(documents=pdf_documents)
            #nodes = pipeline.run(documents=pdf_documents, num_workers=4)
  
    save_progress_to_file(i, progress_file)
    
    if i % max_file_to_process == 0:
        break


Processing HTML files:   0%|          | 5/10565 [00:12<7:15:17,  2.47s/it]


KeyboardInterrupt: 

In [29]:
# the index is the key component to search the documents
index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store
)
q1 ="welche Rolle spielt der Bundesausschuss?"
q2 = 'was soll die Arzneimittelleitlinie zeigen?'

In [30]:
# retrieve and synthesize using out of the box query engine
query_engine = index.as_query_engine(llm=llm)
query = q1
response = query_engine.query(query)
print(f'question: {query}\nresponse:{response}\n')


question: welche Rolle spielt der Bundesausschuss?
response:The Gemeinsamer Bundesausschuss (Federal Joint Committee) plays a role in reviewing and approving decisions related to pharmaceuticals, including the evaluation of new medications with innovative active substances.



In [9]:
# just retrieve from the vectorstore based on similarity to query
retriever = index.as_retriever(
    similarity_top_k=1
)
response = retriever.retrieve(q1)
print("Retriever output:\n")
for node in response:
    print("node score:", node.score)
    print("node text:", node.text)
    print("node metadata:", node.metadata)
    print("\n")  


Retriever output:

node score: 0.7332306
node text: REFERAT  213 
BEARBEITET VON  Walter Schmitz 
HAUSANSCHRIFT  Rochusstraße 1, 53123 Bonn 
POSTANSCHRIFT  53107 Bonn 
TEL  +49 (0)228 99 441-3103 
FAX  +49 (0)228 99 441-4924 
E-MAIL  walter.schmitz@bmg.bund.de 
INTERNET  www.bmg.bund.de  
  Bonn, 22. Dezember 2011  
AZ  213 - 21432 - 01 Bundesministerium für Gesundheit, 53107 Bonn 
  
Gemeinsamer Bundesausschuss 
Wegelystraße 8 
10623 Berlin 
 
  
vorab per Fax: 030 – 275838105 
 __ 
__  
    Beschluss des Gemeinsamen Bundesausschusses gem. § 91 SGB V vom 24.11.2011 hier: 
Änderung der Arzneimittel-Richtlinie (AM-RL): 
 Abschnitt O – Bewertung des Zusatznutzens von Arzneimitteln mit neuen 
Wirkstoffen 
 Anlage XII – Beschlüsse über die Nutzenbewertung von Arzneimitteln mit 
neuen Wirkstoffen nach § 35a SGB V 
 
 Sehr geehrte Damen und Herren,  der von Ihnen gemäß § 94 SGB V vorgelegte o.a. Beschluss vom 24.11.2011 zur Änderung der Arzneimittel-Richtlinie wird nicht beanstandet.  Mit fr

In [10]:
# filter the context using the metadata fields
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
filters = Filter(
    should=[
        Filter(
            must=[
                FieldCondition(
                    key="section",
                    match=MatchValue(value="Beschlusstext"),
                ),
                FieldCondition(
                    key="file_type",
                    match=MatchValue(value="pdf"),
                ),
            ]
        )
    ]
)

retriever = index.as_retriever(
                vector_store_kwargs={"qdrant_filters": filters}, 
                similarity_top_k=3)
response = retriever.retrieve(q1)
print("Retriever output:\n")
for idx, node in enumerate(response):
    print(f"node {idx}\n-----------------")
    
    print("score:", node.score)
    print("text:", node.text)
    print("metadata:", node.metadata)
    print("\n")
    

Retriever output:

node 0
-----------------
score: 0.7131245
text: Beschluss   
 des Gem einsamen Bundesausschuss es 
über eine  Nicht -Änderung  des Beschlusses vom 
15. Oktober 2015 zur Änderung der 
Rehabilitations -Richtlinie : Vereinfachung 
Verordnungsverfahren/Qualifikationsanforderun
gen 
Vom 18. Februar 2016  
Der Gemeinsame Bundesausschuss  (G-BA) hat in seiner Sitzung am 18. Februar 2016 zum 
Beschluss vom 15. Oktober  2015 zur Änderung der Richtlinie über Leistungen zur 
medizinischen Rehabilitation (Rehabilitations -Richtlinie) in der Fassung vom 16. März 2004  
(BAnz. S. 6769 ), zuletzt geändert am 17. April 2014 (BAnz AT 26.06.2014  B 4), beschlossen:  
 
I. Der Beschluss wird nicht geändert.  
 
II. Dieser Beschluss tritt mit Beschlussfassung  in Kraft.  
 
Die T ragenden Gründe werden auf den  Internetseite n des Gemeinsamen 
Bundesausschusses unter www.g -ba.de  veröffentlicht.  
 Berlin, den 18. Februar 2016 
Gemeinsamer Bundesausschuss  
gemäß § 91 SGB  V 
Der Vorsi

In [11]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.retrievers import VectorIndexRetriever

class RAGQueryEngine(CustomQueryEngine):
    """RAG Query Engine."""
    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        response_obj = self.response_synthesizer.synthesize(query_str, nodes)
        return response_obj

query_engine = RAGQueryEngine(
        retriever = VectorIndexRetriever(
            index = index,
            similarity_top_k = 3,
        ),
        response_synthesizer = get_response_synthesizer(
            response_mode="tree_summarize",
            verbose=True,
        )
)

response = query_engine.query(q1)
print(response) 
    

1 text chunks after repacking
The Gemeinsamer Bundesausschuss (G-BA) plays a crucial role in evaluating the added value of medicinal products with new active substances. According to § 35a Abs. 1 SGB V, the G-BA assesses the benefit of reimbursable medicinal products with new active substances, including the evaluation of the additional benefit compared to conventional therapy, the extent of the additional benefit, and its therapeutic significance.

The G-BA also decides on the added value of medicinal products with new active substances according to § 35a Abs. 3 SGB V. This decision is part of the Arzneimittel-Richtlinie (Medicinal Products Directive) and has an impact on the prescribing doctor's decision-making process.

In addition, the G-BA provides clear guidelines for the prescribing doctor in its decisions, outlining which findings are relevant for prescription purposes and which aspects should be considered when prescribing a medicinal product.


In [12]:

from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate

# define modules
question = q1
prompt_str = "Please answer the following question: {question}"
prompt_tmpl = PromptTemplate(prompt_str)
retriever = index.as_retriever(similarity_top_k=3)
summarizer = TreeSummarize(llm=llm)

In [13]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle

import pandas as pd
from IPython.display import display, HTML
def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))

def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {
            "Score": node.score, 
            "Text": node.node.get_text(), 
            "Section": node.node.metadata["section"]
        }
        result_dicts.append(result_dict)
    pretty_print(pd.DataFrame(result_dicts))


In [14]:
query_bundle = QueryBundle(q1)
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10
)
retrieved_nodes = retriever.retrieve(query_bundle)
visualize_retrieved_nodes(retrieved_nodes)


,Score,Text,Section
0,0.733231,"REFERAT 213 BEARBEITET VON Walter Schmitz HAUSANSCHRIFT Rochusstraße 1, 53123 Bonn POSTANSCHRIFT 53107 Bonn TEL +49 (0)228 99 441-3103 FAX +49 (0)228 99 441-4924 E-MAIL walter.schmitz@bmg.bund.de INTERNET www.bmg.bund.de Bonn, 22. Dezember 2011 AZ 213 - 21432 - 01 Bundesministerium für Gesundheit, 53107 Bonn Gemeinsamer Bundesausschuss Wegelystraße 8 10623 Berlin vorab per Fax: 030 – 275838105 __ __ Beschluss des Gemeinsamen Bundesausschusses gem. § 91 SGB V vom 24.11.2011 hier: Änderung der Arzneimittel-Richtlinie (AM-RL): Abschnitt O – Bewertung des Zusatznutzens von Arzneimitteln mit neuen Wirkstoffen Anlage XII – Beschlüsse über die Nutzenbewertung von Arzneimitteln mit neuen Wirkstoffen nach § 35a SGB V Sehr geehrte Damen und Herren, der von Ihnen gemäß § 94 SGB V vorgelegte o.a. Beschluss vom 24.11.2011 zur Änderung der Arzneimittel-Richtlinie wird nicht beanstandet. Mit freundlichen Grüßen Im Auftrag Dr. Tautz Dienstgebäude Bonn-Duisdorf, Rochusstraße 1: Bushaltest elle Rochusstr./Bundesministerien (636, 637, 638, 800, 845) Dienstgebäude Bonn-Duisdorf, Heilsbachstraße 18: Bahnhof Bonn-Duisdorf, ca. 5 Min. Fußw eg (Bn.-Hbf.: Gleis 5, RB 23 Richtung E uskirchen)",Prüfung gem. § 94 SGB V durch das BMG
1,0.724582,"nisformenein.FürdiePrüfungdieserVoraus-setzungistdeshalbfristgerechtdasOriginaleinerbeglaubigtenKopiedeserteiltenPatentsdeneinzureichendenUnterlagenbeizufügen.Anderenfallsmussdavonausgegangenwerden,dassfürdenWirkstoffdesbetreffendenArzneimittelskeingültigesWirkstoffpatentvorliegt,mitderFolge,dassdieMöglichkeit,dasArzneimittelvoneinerFestbetragsgruppenbildungwegeneinertherapeutischenVerbesserungauszunehmen,nichtbesteht.StellungnahmenzudiesenEntwürfeneinschließlichLiteratursowieLiteratur-bzw.Anlagenverzeichnissind–soweitnichtausdrücklichimOriginalangefordert–inelektronischerForm(z.B.perCD/DVDoderperE-Mail)alsWord-Dateibzw.dieLiteraturalsPDF-Dateibiszum2.Mai2016zurichtenan:GemeinsamerBundesausschussAbteilungArzneimittelWegelystraße810623BerlinE-Mail:festbetragsgruppen@g-ba.deBetroffenepharmazeutischeUnternehmen,dienichtMitgliederderobengenanntenVerbändesind,erhaltendieEnt-würfesowiedieTragendenGründebeiderGeschäftsstelledesGemeinsamenBundesausschusses.DieBeschlüsseunddieTragendenGründekönnenaufdenInternetseitendesGemeinsamenBundesausschussesunterwww.g-ba.deeingesehenwerden.Berlin,den8.März2016GemeinsamerBundesausschussgemäß§91SGBVDerVorsitzendeProf.Heckenwww.bundesanzeiger.deBekanntmachungVeröffentlicht am Montag, 4. April 2016BAnz AT 04.04.2016 B5Seite 1 von 1",Bekanntmachung des Stellungnahmeverfahrens im Bundesanzeiger
2,0.719430,"2 1. Rechtsgrundlagen Nach § 35a Abs. 1 SGB V bewertet der Gemeinsame Bundesausschuss (G -BA) den Nutzen von erstattungsfähigen Arzneimitteln mit neuen Wir kstoffen. Hierzu gehört insbesondere die Bewertung des Zusatznutzens gegenüber der zwec k-mäßigen Vergleichstherapie, des Ausmaßes des Z usatznutzens und seiner the-rapeutischen Bedeutung. Nach § 35a Absatz 3 SGB V beschließt der G-BA über die Nutzenbewertung von Arzneimitteln mit neuen Wirkstoffen. Der Beschluss ist Teil der Richt linie nach § 92 Absatz 1 Satz 2 Nummer 6 SGB V (Arzneimittel -Richtlinie) . 2. Eckpunkte der Entscheidung Der Beschluss über die Nutzenbewertung nach § 35a Abs.3 SGB V ist eine Fes t-stellung über die Zweckmäßigkeit des Arzneimittels im Sinne des § 12 SGB V und entfaltet daher Wirkung für den Vertragsarzt bei der Verordnung. Dem Ve r-tragsarzt soll aber nicht die Kenntnis aller Einzelheiten der Nutzenbewertung z u-gemutet werden. Daher soll der Gemeinsame Bundesausschuss in seinem B e-schluss ausdrüc klich klarstellen, welche Feststellungen bei der Verordnung zu beachten und daher Teil der Arzneimittelrichtlinien sind (vgl. BT -Drucks. 17/2413, S.20) . Diesen Anforderungen Rechnung tragend werden in § 51 in Form einer nicht a b-schließenden Aufzählung die für die Verordnung eines Arzneimittels nach § 12 SGB V relevanten Gesichtspunk te aufgeführt. Soweit mög lich soll in der 